<a href="https://colab.research.google.com/github/arehfeldt/Covid19-Config-Generator/blob/main/Aaron_Rehfeldt_CSE5522_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CSE 5522 Hands-on #1: Linear Regression**

The goals of today's exercise are to familarize you with:


*   Gradient descent
*   Multivariate regression
*   Data exploration

**END OF CLASS GOAL:** Submit a link to your notebook (Share > Get Sharable Link) in Carmen so I can see how far you got.  This should be submitted in a group assignment page on Carmen.

**Part 0: Initial setup**

**0.0:** If none of your team members are familar with python this will be difficult to accomplish - you may want to split up and join different groups.

**0.1:** Make a copy of this page in your google drive so that you can edit it. Share the copied page with your teammates. At the end of class, share a URL and submit (so I can see how far you got).  This will count as the participation grade for all members.

**0.2:** While not completely necessary for this assignment, you may want to familiarize yourself with the following packages: [numpy](https://numpy.org), [scikit-learn](https://scikit-learn.org), [pandas](https://pandas.pydata.org), [matplotlib](https://matplotlib.org).

---
---


**Part 1: Univariate Linear Regression & Gradient Descent**

In class, we discussed how minimizing the Mean Squared Error criterion can lead to one solution of "best fit" for linear regression.  We looked at a dataset that related housing price and square footage in Columbus.  Your first task will be to replicate the regression algorithm that was demonstrated in class. Some of the more straightforward tasks will be given to you in code; some will need to be filled in.

**1.0:** Set up the environment (you can click on the play button below to import the appropriate modules).

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

**1.1** Read the data from GitHub into a pandas dataframe.

In [ ]:
ColumbusHousingUrl='https://github.com/efosler/cse5522data/raw/master/columbus_price_vs_sf.csv'
colhouse_dataframe=pd.read_csv(ColumbusHousingUrl)

**1.2** Print out the top of the dataframe to make sure that the data loaded correctly.  It should be a data table with two columns (price, sf), and 5 rows, with the first row 220000,2240.

In [ ]:
colhouse_dataframe.head()

,price,sf
0,220000,2240
1,171900,1720
2,201500,2786
3,140000,1040
4,159000,2038


**1.3:** At this point you may want to select columns from the dataframe; this is optional but may make debugging a bit easier as dataframes are not really arrays but are iterables.  Personally I like to use numpy arrays but you can choose to use standard python arrays if you wish.

In [ ]:
sf=np.array(colhouse_dataframe['sf'],dtype='float').reshape(-1,1)
price=np.array(colhouse_dataframe['price'],dtype='float').reshape(-1,1)
sf.shape
price.shape

(222, 1)

**1.4:** Now the real work begins. Write a function that, given the dataset (sf,price) and a current set of weights, returns the gradient of the mean squared error.  You may wish to refer to the class notes on the MSE gradient.

In [ ]:
# computeMSEBatchGradient:
#   weights - vector of weights (univariate linear = 2 weights)
#   features - vector (or matrix) of feature values
#   targets - vector of target values, same length as features
#
#   returns average gradient over the batch of features
def computeMSEBatchGradient(weights,features,targets):

  # insert calculation of gradient here
  gradient = [0, 0]
  for i in range(len(features)):
    gradient[0] += targets[i] - (weights[1] * features[i] + weights[0])
    gradient[1] += features[i] * (targets[i] - (weights[1] * features[i] + weights[0]))
  gradient[0] *= -2/len(features)
  gradient[1] *= -2/len(features)
  return gradient

**1.5:** Next, write the function that calculates a weight update based on the gradient and learning rate.

In [ ]:
# computeGradientDescentWeightUpdate:
#   weights - vector initial weights
#   features - vector/matrix of features
#   targets - vector of target values
#   learning_rate - step length for update
def computeGradientDescentWeightUpdate(weights, features, targets, learning_rate):
  # insert calculation of weight update here
  gradient = computeMSEBatchGradient(weights, features, targets)
  weights[0] = weights[0] - learning_rate * gradient[0]
  weights[1] = weights[1] - learning_rate * gradient[1]
  return weights


**1.6:** Now write the outer wrapper that initializes the weights and learning rate and runs the gradient update in a loop.  You can choose to run for a fixed number of iterations, or test for convergence by seeing if the weights change more than some small amount.  Print out the weights at the end.   *Note: if your code runs for more than a minute then you probably have a diverging set of weights.*


In [ ]:
# insert wrapper code here
weights = [0, 0]
learning_rate = 0.001
for i in range(1000):
  weights = computeGradientDescentWeightUpdate(weights, sf, price, learning_rate)
weights


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in multiply
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in subtract
  # Remove the CWD from sys.path while we load stuff.


[array([nan]), array([nan])]

**1.7:** Compare your results against a known linear regression algorithm from *scikit-learn*.  Do you get the same values for slope and intercept?




In [ ]:
from sklearn.linear_model import LinearRegression

lin_model=LinearRegression()
lin_model.fit(sf,price)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# print slope
lin_model.coef_

array([[110.08078024]])

In [ ]:
# print intercept
lin_model.intercept_

array([-39486.50513007])

**1.8:** At this point, go back to 1.6 and try tuning the learning rate.  Can you notice any differences in the algorithm's performance?  If your algorithm didn't converge previously, you may want to adjust the scale of your data (maybe make them fall in roughly the same range), or have multiple learning rates.  


---


*Hint: you may want to print the weights every 1000 updates or so to see how the weights change over time.  If you want to get really tricky you can try to reimplment the movie we showed in class, but probably beyond the scope of this exercise.*

---

One trick that ML practitioners often do is to *normalize* the data by subtracting the mean and dividing by the standard deviation; this means that you can usually use a learning rate in roughly the same ballpark each time.   

---
---

**Part 2: Multivariate regression**

In this part of the hands-on class, you'll explore multivariate regression.  We'll use the *scikit-learn* linear regression tool (although you're welcome to extend your code above if you wish).  The primary reason for this is that the sklearn routines will handle some of the normalization issues for you.

Some parts of this tutorial were based on [Argawal, Linear Regression on Boston Housing Dataset](https://towardsdatascience.com/linear-regression-on-boston-housing-dataset-f409b7e4a155).

**2.0:** We'll be looking at the Boston Housing dataset which is built in to sklearn, but has more variables. Load the dataset in:

In [ ]:
from sklearn.datasets import load_boston
boston_dataset=load_boston()

**2.1** You can see a description of the dataset by printing the dataset's DESCR attribute (set by the sklearn.dataset package).

In [ ]:
print(boston_dataset.DESCR)

**2.2:** We can convert this data into a pandas dataframe and then look at the first few rows in tabular form.

In [ ]:
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston['MEDV']=boston_dataset.target
boston.head()

**2.3:** At this point, we want to figure out what variables are good predictors of the median value of a house in Boston.  We can find the relationship between single variables by looking at the correlation between variables.  

---

*Side note: the correlation between two vectors of variables x and y is*
```
1/(n-1) sum x*y
```
*assuming that x and y have zero mean and a standard gaussian distribution (which you can achieve by subtracting the mean and dividing by the standard deviation).  If x and y always share the same sign, then this value will be positive; if x and y are always opposite sign then the correlation is negative.*

---

The sklearn.datasets class has a built in correlation function, which we can use to compute the correlation between each pair of variables.  We can also use the seaborn package to plot a nice heatmap of these variables.

**What's the most correlated variable with MEDV?**


In [ ]:
import seaborn as sb

correlation_matrix = boston.corr().round(2)
plt.figure(figsize=(10,8))
sb.heatmap(data=correlation_matrix, annot=True, center=0.0, cmap='coolwarm')

Note that the most correlated variable is LSTAT (at -0.74) and not RM (0.70) - while the correlation is negative, it just means that there is an inverse relationship between LSTAT and MEDV, but LSTAT should be the single best predictor.

**2.4:** Now, we kind of cheated there, since we figured out the correlation on the entire dataset.  A better practice is to randomly select training and test sets, fit the data on the training set, and then evaluate on the test set.  Let's see if LSTAT really is a better predictor than RM of MEDV.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# first, set up dataframe for all variables
boston_totaldata = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston_totaltarget = boston_dataset.target

boston_traindata, boston_testdata, boston_traintarget, boston_testtarget = train_test_split(boston_totaldata,
                                                                                           boston_totaltarget,
                                                                                           test_size=0.2,
                                                                                           random_state=5)

In [ ]:
boston_traindata.shape

In [ ]:
boston_testdata.shape

In [ ]:
# subselect LSTAT data, train model
#   note the double square brackets - need 2-dimensional array to come out of selection
lstat_traindata=boston_traindata[['LSTAT']]
lstat_testdata=boston_testdata[['LSTAT']]
lin_model = LinearRegression()
lin_model.fit(lstat_traindata,boston_traintarget)

In [ ]:
lstat_testpredict=lin_model.predict(lstat_testdata)
rmse = (np.sqrt(mean_squared_error(boston_testtarget,lstat_testpredict)))
r2 = r2_score(boston_testtarget,lstat_testpredict)
print('Test RMSE = {}, Test R2 = {}'.format(rmse,r2))

**2.4a:** R2 is the amount of variation explained by the model, and can range from 0 (no variance explained) to 1 (perfect explanation). Notice that R2 is much lower than the correlation coefficient.  Why? (There are 2 reasons.)

Let's do it again for RM (this time, you calculate it).

In [ ]:
# fill in code to calculate RMSE, R2 for RM variable


(Note: answers for why R2 is lower that correlation is that (a) it's roughly the square of correlation, and also (b) we calculated correlation on the entire training set; R2 was calculated on test only.)

**2.5:** At this point, you can start to look at how to predict from multiple variables.  What are the two best variables in predicting as measured by R2?

Are the results better than predicting using all variables?

In [ ]:
# Rebuild your training set to look at two variable prediction.

---
---
**Part 3: Predicting Gene Expression**

There are ~20,000 genes in the human genome. Each one of them is transcribed to mRNA and then translated to proteins which carry on various tasks inside our body. We can measure the amount of 20,000 mRNA expressed in samples collected from different organs. This collection is called _gene expression profile_.

Although our genome is the same across all cell types, the gene expression profile is different because each organ needs different proteins for its survival. One of the regulatory mechanisms which controls the expression level in each cell type is microRNA (miR). MicroRNAs are small molecules which attach to mRNAs and prevent them from translation to proteins and also degrade them.

So if miR A targets mRNA B when A increases B decreases. Our goal is to predict mRNA levels (gene expression profile) using 21 miR features. Note that each of the 20,000 expression levels can be a response of regression with 21 features. To simplify, we have selected a few genes to predict their expression.

This part of the exercise is deliberately more open-ended.

**Instructions:** Load the data from GitHub (example code given).  Use the Boston Housing example to *randomly* break the data into an 80/20 split across the 8895 examples.  Pick a few random examples from the well expressed set and from the poorly expressed set and train a linear regression predictor for each mRNA.  Describe the differences you see across the two sets.

In [ ]:
miR_URL='https://raw.githubusercontent.com/efosler/cse5522data/master/miRScore-x.csv'
miR_df=pd.read_csv(miR_URL)
# remove row names from the frist column, rows=samples, cols=miR features
miR = np.array(miR_df)[:,1:-1]

mRNA_well_URL='https://raw.githubusercontent.com/efosler/cse5522data/master/mRNA-y-well-explained.csv'
mRNA_well_df=pd.read_csv(mRNA_well_URL)
# remove label column, transpose with rows=samples, cols=mRNA
mRNA_well = np.transpose(np.array(mRNA_well_df)[:,1:8896])

mRNA_poor_URL='https://raw.githubusercontent.com/efosler/cse5522data/master/mRNA-y-poor-explained.csv'
mRNA_poor_df=pd.read_csv(mRNA_poor_URL)
# remove label column, transpose with rows=samples, cols=mRNA
mRNA_poor = np.transpose(np.array(mRNA_poor_df)[:,1:8896])

In [ ]:
# Carry on from this point on your own, using the Boston Housing problem as a guide.
